In [1]:
!pip install streamlit pyngrok --quiet


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 489.8 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.9/9.9 MB 22.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 64.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 7.5 MB/s eta 0:00:00


In [2]:
from pyngrok import ngrok
ngrok.set_auth_token("30rbzJpbGd0hZ09DKkCkNabczXN_6oE5TfmTeE3i2SGr9XNiu")  # <-- optional


In [5]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score
import pickle

# Load dataset
df = pd.read_csv("spam.csv", encoding='latin-1')
# The columns are already named 'Category' and 'Message'
# df = df[['v1', 'v2']] # No need to select and rename columns
# df.columns = ['Category', 'Message'] # No need to rename columns

# Encode labels
df['label'] = df['Category'].map({'ham': 0, 'spam': 1})

# Split data
X_train, X_test, y_train, y_test = train_test_split(df['Message'], df['label'], test_size=0.2)

# Vectorize text
vectorizer = CountVectorizer()
X_train_vec = vectorizer.fit_transform(X_train)
X_test_vec = vectorizer.transform(X_test)

# Train classifier
model = MultinomialNB()
model.fit(X_train_vec, y_train)

# Accuracy
y_pred = model.predict(X_test_vec)
print("Accuracy:", accuracy_score(y_test, y_pred))

# Save model and vectorizer
pickle.dump(model, open("spam_model.pkl", "wb"))
pickle.dump(vectorizer, open("vectorizer.pkl", "wb"))

Accuracy: 0.9829596412556054


In [6]:
%%writefile app.py
import streamlit as st
import pickle

# Load model and vectorizer
model = pickle.load(open("spam_model.pkl", "rb"))
vectorizer = pickle.load(open("vectorizer.pkl", "rb"))

# Streamlit app
st.title("📧 Email Spam Classifier")

msg = st.text_area("Enter your email message here")

if st.button("Predict"):
    data = vectorizer.transform([msg])
    prediction = model.predict(data)

    if prediction[0] == 1:
        st.error("🚫 Spam Email")
    else:
        st.success("✅ Not Spam")


Writing app.py


In [8]:
from pyngrok import ngrok
import time

# Kill any previous Streamlit processes
!pkill streamlit

# Run Streamlit app
!streamlit run app.py &>/content/logs.txt &

# Wait for app to load
time.sleep(5)

# Create tunnel
public_url = ngrok.connect(addr=8501, proto='http')
print("🔗 Streamlit app is live at:", public_url)

🔗 Streamlit app is live at: NgrokTunnel: "https://4702e200361c.ngrok-free.app" -> "http://localhost:8501"
